In [ ]:
# DataSci GPU Tutor (RAG + LLM + Game Mode)
# Includes: FAISS RAG system, fallback to Falcon-7B-Instruct, and full game mode

# --- RAG Utilities ---
def read_urls_from_txt(path):
    with open(path, "r") as f:
        return [line.strip() for line in f if line.strip()]

def fetch_text_from_url(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (RAG Tutor Bot)'}
        resp = requests.get(url, headers=headers, timeout=15)

        # Filter out non-text content types        content_type = resp.headers.get("Content-Type", "")
        if not content_type.startswith("text/") and "html" not in content_type:
            print(f"⚠️ Skipping non-text URL: {url} (type={content_type})")
            return ""

        # Parse HTML content safely
        soup = BeautifulSoup(resp.content, 'html.parser')
        for tag in soup(['script', 'style', 'header', 'footer', 'nav', 'aside']):
            tag.decompose()
        return soup.get_text(separator="\n").strip()
    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")
        return ""

def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size - overlap)]

def load_all_chunks(urls):
    all_chunks = []
    for url in urls:
        txt = fetch_text_from_url(url)
        if txt:
            all_chunks.extend(chunk_text(txt))
    return all_chunks

def embed_and_index(chunks):
    vecs = embedder.encode(chunks, show_progress_bar=True, convert_to_numpy=True)
    dim = vecs.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(vecs))
    return index

def query_rag(query, index, chunks, k=3):
    if index is None or not chunks:
        return "", ""
    q_vec = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(np.array(q_vec), k)
    return "\n".join([chunks[i] for i in I[0]]), "Custom Knowledge Base"

def load_cache():
    if os.path.exists(CHUNK_FILE) and os.path.exists(INDEX_FILE):
        with open(CHUNK_FILE, "rb") as f:
            chunks = pickle.load(f)
        index = faiss.read_index(INDEX_FILE)
        return chunks, index
    return None, None

def save_cache(chunks, index):
    with open(CHUNK_FILE, "wb") as f:
        pickle.dump(chunks, f)
    faiss.write_index(index, INDEX_FILE)

def build_or_load_rag():
    chunks, index = load_cache()
    if not chunks or index is None:
        urls = read_urls_from_txt(URLS_FILE)
        chunks = load_all_chunks(urls)
        index = embed_and_index(chunks)
        save_cache(chunks, index)
    return chunks, index

chunks, index = build_or_load_rag()

# --- Smart Tutor Answer: RAG + Fallback ---
DOC_LINKS = {
    # Core Libraries
    "pandas": "https://pandas.pydata.org/docs/",
    "scikit-learn": "https://scikit-learn.org/stable/",
    "rapids": "https://rapids.ai/",
    "cudf": "https://docs.rapids.ai/api/cudf/stable/",
    "cupy": "https://docs.cupy.dev/en/stable/",
    "pytorch": "https://pytorch.org/docs/stable/",
    "tensorflow": "https://www.tensorflow.org/",
    "cuml": "https://docs.rapids.ai/api/cuml/stable/",

    # Data Science
    "dsml_pdf": "https://people.smp.uq.edu.au/DirkKroese/DSML/DSML.pdf",
    "awesome_datascience": "https://github.com/academic/awesome-datascience?tab=readme-ov-file",
    "gfg_ds_beginners": "https://www.geeksforgeeks.org/data-science/data-science-for-beginners/",

    # GPU Acceleration
    "cudf_pandas": "https://rapids.ai/cudf-pandas/",
    "polars_gpu": "https://rapids.ai/polars-gpu-engine/",
    "cuml_accel": "https://rapids.ai/cuml-accel/",
    "nx_cugraph": "https://rapids.ai/nx-cugraph/",
    "hagedorn2022": "https://proceedings.mlr.press/v185/hagedorn22a/hagedorn22a.pdf",
    "nvidia_tesla_whitepaper": "https://www.nvidia.com/docs/io/116711/sc11-nv-tesla.pdf",
    "cuda_blog": "https://developer.nvidia.com/blog/tag/cuda/",
    "libcxxgpu_pdf": "https://www.seas.upenn.edu/~delozier/docs/libcxxgpu.pdf",
    "gpu_accel_slides": "https://dlsyscourse.org/slides/12-gpu-acceleration.pdf",
    "gpucad_paper": "https://yibolin.com/publications/papers/GPUCAD_ICCAD2020_Lin.pdf",
    "gpucad_slides": "https://yibolin.com/publications/papers/GPUCAD_ICCAD2020_Lin.slides.pdf",
    "spie_gpu_sample": "https://www.spiedigitallibrary.org/samples/SL34.pdf",
    "gpu_to_web": "https://www.khronos.org/assets/uploads/developers/library/2012-the-graphical-web/GPU-to-the-web_Sep2012.pdf",

    # RAG Retrieval
    "langgraph_agentic_rag": "https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/",

    # Basics
    "gpu_compare": "https://www.nvidia.com/en-us/geforce/graphics-cards/compare/",
    "gpu_hierarchy": "https://www.tomshardware.com/reviews/gpu-hierarchy,4388.html"
}

import plotly.graph_objects as go
import base64
from io import BytesIO

# --- Smart Tutor Answer: RAG + Fallback ---
DOC_LINKS = {
    "pandas": "https://pandas.pydata.org/docs/",
    "scikit-learn": "https://scikit-learn.org/stable/",
    "rapids": "https://rapids.ai/",
    "cudf": "https://docs.rapids.ai/api/cudf/stable/",
    "cupy": "https://docs.cupy.dev/en/stable/",
    "pytorch": "https://pytorch.org/docs/stable/",
    "tensorflow": "https://www.tensorflow.org/",
    "cuml": "https://docs.rapids.ai/api/cuml/stable/",
}

related_examples = {
    "gpu": ["What is CUDA?", "cuDF vs pandas?", "cuML for ML tasks?"],
    "pandas": ["How to groupby in pandas?", "pandas vs cuDF?", "Time series in pandas"],
    "scikit-learn": ["What is train_test_split?", "GridSearchCV usage", "PCA in sklearn"]
}

def suggest_related(query):
    suggestions = []
    for keyword, rel_list in related_examples.items():
        if re.search(rf"\\b{keyword}\\b", query, re.I):
            suggestions = rel_list
            break
    if suggestions:
        bullets = "".join([f"- {q}<br>" for q in suggestions])
        return f"<br><br><b>🔎 You might also ask:</b><br>{bullets}"
    return ""

def generate_example_plot():
    x = [100, 500, 1000, 5000, 10000]
    cpu_times = [0.5, 1.8, 3.2, 15.0, 30.0]
    gpu_times = [0.2, 0.5, 0.8, 2.5, 5.0]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=cpu_times, mode='lines+markers', name='CPU'))
    fig.add_trace(go.Scatter(x=x, y=gpu_times, mode='lines+markers', name='GPU'))
    fig.update_layout(title='CPU vs GPU Execution Time',
                      xaxis_title='Input Size',
                      yaxis_title='Time (s)',
                      legend_title='Processor')

    buffer = BytesIO()
    fig.write_image(buffer, format='png')
    img_base64 = base64.b64encode(buffer.getvalue()).decode()
    return f'<img src="data:image/png;base64,{img_base64}" style="width:100%;max-width:600px;">'

#TO-DO
#def polish_response(text):
    #prompt = f"Polish this answer to be clear, helpful, and concise like a good AI tutor. Avoid repetition:\n\n{text}\n\nPolished Answer:"
def polish_response(text):
    prompt = f"""
You're an expert socratic AI tutor for Data Science and GPU Acceleration. Take the content below and improve it into a well-structured explanation that is:

- Friendly and clear
- Easy for students to understand
- Broken into logical sections. Ask questions before you answer. Make them think.
- Uses analogies/examples when helpful
- Suggests any charts or visualizations if relevant
- Includes a short summary or takeaway at the end

Original content:
-------------------------------
{text}
-------------------------------

Now improve and rewrite it in tutor style:
"""
    try:
        raw = llm_pipeline(prompt)[0]['generated_text']
        split = raw.split("Now improve and rewrite it in tutor style:")
        return split[-1].strip() if len(split) > 1 else raw.strip()
    except Exception as e:
        print(f"❌ polish_response failed: {e}")
        return "Sorry, I couldn’t process this content right now."
    
def smart_tutor_answer(query):
    context, source = query_rag(query, index, chunks)

    def is_garbage(text):
        non_ascii = sum(1 for c in text if ord(c) > 126 or ord(c) < 9)
        return len(text) < 80 or non_ascii / len(text) > 0.2

    if not context or is_garbage(context):
        print("⚠️ RAG returned bad or short text. Falling back to LLM.")
        prompt = f"You're a concise, helpful AI tutor for data science and GPU acceleration. Answer this clearly:\n\nQ: {query}\nA:"
        result = llm_pipeline(prompt)[0]['generated_text']
        answer = result.strip().split("A:")[-1].strip()

        doc_link = ""
        for k, v in DOC_LINKS.items():
            if re.search(rf"\\b{k}\\b", query, re.I):
                doc_link = f"\n\n🔗 [Official {k.title()} Docs]({v})"
                source = v
                break
        graph = generate_example_plot() if "gpu" in query.lower() or "benchmark" in query.lower() else ""
        return answer + doc_link + suggest_related(query) + "<br><br>" + graph, source if source else "LLM (fallback)"

    else:
        polished = polish_response(context.strip())
        graph = generate_example_plot() if "gpu" in query.lower() or "benchmark" in query.lower() else ""
        extras = suggest_related(query)
        return f"{polished}<br><br>{extras}<br><br>{graph}", source if source else "Custom Knowledge Base"

def generate_flashcards_from_rag(query, k=3):
    context, source = query_rag(query, index, chunks)

    if not context or len(context.strip()) < 100:
        return [{"front": "Could not find content", "back": "Try another topic."}]

    prompt = f"""
You're an expert AI tutor for Data Science and GPU Acceleration. From the text below, extract {k} flashcards. Each flashcard should have:
- A question on the front (about a key concept)
- A short, accurate answer on the back

Text:
--------------------
{context}
--------------------

Now generate {k} flashcards in this JSON format:
[
  {{"front": "What is ...?", "back": "It is ..."}},
  ...
]
"""
    try:
        raw = llm_pipeline(prompt)[0]['generated_text']
        match = re.search(r"\[(\s*\{.*?\}\s*,?\s*)+\]", raw, re.DOTALL)
        if match:
            return eval(match.group(0))  # Caution: use `ast.literal_eval()` in production
    except Exception as e:
        print(f"❌ Flashcard generation failed: {e}")

    return [{"front": "Could not parse flashcards", "back": "Try a different query."}]  

with gr.Blocks(theme=gr.themes.Soft(), title="M^6 AI Tutor") as app:
    gr.Markdown("""
    <div style='text-align:center; background:linear-gradient(90deg,#fff1c1,#c1e7ff,#e1ffc1); border-radius:15px; padding:10px 0; margin-bottom:8px; color:#000;'>
      <h1 style='color:#000;'>🧠 M^6 AI Tutor</h1>
      <h3 style='color:#000;'>Switch between <span style='color:#0a88a4'>Tutor Mode</span> and <span style='color:#0a88a4'>Game Mode</span> for interactive learning!</h3>
      <p style='color:#111; font-weight:700; font-size:1.05em;'>
        <b>Learn, Play, and Benchmark real data science and GPU acceleration!</b>
      </p>
    </div>
    """)

    with gr.Tabs():
        with gr.Tab("Tutor Mode"):
            tutor_query = gr.Textbox(label="Ask about Data Science or GPU topics", placeholder="e.g., What is RAPIDS?")
            tutor_answer_box = gr.Markdown(label="AI Tutor's Answer", value="")
            tutor_source_box = gr.Markdown(label="Source of Answer", value="")
            tutor_btn = gr.Button("🚀 Get Tutor Help")
            def tutor_handler(q):
                answer, source = smart_tutor_answer(q)
                return answer, f"**Source:** {source}"
            tutor_btn.click(fn=tutor_handler, inputs=tutor_query, outputs=[tutor_answer_box, tutor_source_box])
            tutor_query.submit(fn=tutor_handler, inputs=tutor_query, outputs=[tutor_answer_box, tutor_source_box])

        with gr.Tab("Game Mode"):
            user_state = SimpleNamespace(points=0, level=1)
            def update_level():
                user_state.level = min(10, 1 + user_state.points // 10)
                bar = f"""<b>⭐ {user_state.points} points — Level {user_state.level}</b><br>
                <div style='background:#eee;border-radius:8px;width:100%;height:18px;'>
                <div style='background:linear-gradient(90deg,#fceabb,#f8b500);height:18px;width:{(user_state.points % 10)*10}%;border-radius:8px;'></div>
                </div>"""
                return bar
            progress = gr.HTML(update_level())

            with gr.Tabs():
                # --- Flashcards Tab (its own tab, not inside Quiz tab) ---
                with gr.Tab("🃏 Flashcards"):
                    gen_query = gr.Textbox(label="Enter a topic (e.g., cuDF, CUDA, RAPIDS)", placeholder="cuDF vs pandas?")
                    gen_btn = gr.Button("✨ Generate Flashcards")
                    card_front = gr.Markdown()
                    card_back = gr.Markdown(visible=False)
                    flip_btn = gr.Button("🔄 Flip Card", visible=False)
                    reset_btn = gr.Button("🔁 Reset", visible=False)
                    flash_idx = gr.State(0)
                    show_answer = gr.State(False)
                    flashcards_state = gr.State([])

                    def flashcard_from_query(topic):
                        cards = generate_flashcards_from_rag(topic)
                        if not cards:
                            return "No flashcards generated.", "", gr.update(visible=False), gr.update(visible=False), 0, False, cards
                        first = cards[0]
                        return f"**Q:** {first['front']}", "", gr.update(visible=True), gr.update(visible=True), 0, False, cards

                    def flip_generated(idx, show, cards):
                        card = cards[idx % len(cards)]
                        if show:
                            return (
                                f"**Q:** {card['front']}",
                                f"**A:** {card['back']}",
                                gr.update(visible=True),
                                idx,
                                not show,
                                cards
                            )
                        else:
                            return (
                                f"**Q:** {card['front']}",
                                "",  # hide answer
                                gr.update(visible=False),
                                idx,
                                not show,
                                cards
                            )

                    def reset_generated(cards):
                        if not cards:
                            return "", "", 0, False, cards
                        return f"**Q:** {cards[0]['front']}", "", 0, False, cards

                    gen_btn.click(flashcard_from_query, inputs=gen_query, outputs=[card_front, card_back, flip_btn, reset_btn, flash_idx, show_answer, flashcards_state])
                    flip_btn.click(flip_generated, inputs=[flash_idx, show_answer, flashcards_state], outputs=[card_front, card_back, card_back, flash_idx, show_answer, flashcards_state])
                    reset_btn.click(reset_generated, inputs=flashcards_state, outputs=[card_front, card_back, flash_idx, show_answer, flashcards_state])

                # --- Quiz Tab (standalone, not inside Flashcards) ---
                with gr.Tab("❓ Quiz"):
                    QUIZ_TOPICS = [
                        {"name": "Pandas", "color": "#f9e79f"},
                        {"name": "GPU Basics", "color": "#aed6f1"},
                        {"name": "cuDF", "color": "#d5f5e3"},
                        {"name": "Machine Learning", "color": "#fadbd8"},
                        {"name": "CUDA", "color": "#e8daef"}
                    ]

                    def generate_mcqs(topic, n=2):
                        prompt = (
                            f"Generate {n} beginner-friendly multiple-choice questions about {topic}. "
                            f"For each, provide: question, 4 options, and the index (0-based) of the correct answer. "
                            f"Respond as a Python list of dicts with keys: 'question', 'options', 'answer_idx'."
                        )
                        llm_result = llm_pipeline(prompt)[0]['generated_text']
                        try:
                            questions = ast.literal_eval(llm_result)
                            if isinstance(questions, list) and all(
                                'question' in q and 'options' in q and 'answer_idx' in q for q in questions
                            ):
                                return questions
                        except Exception as e:
                            print("LLM parse error:", e)
                        # Fallback if LLM fails
                        return [
                            {"question": f"Sample question about {topic}?", "options": ["A", "B", "C", "D"], "answer_idx": 0},
                            {"question": f"Another sample about {topic}?", "options": ["X", "Y", "Z", "W"], "answer_idx": 1}
                        ]

                    # Topic selection row (colorful boxes)
                    with gr.Row():
                        topic_btns = []
                        for t in QUIZ_TOPICS:
                            btn = gr.Button(t["name"], elem_id=f"topic-{t['name'].lower().replace(' ', '-')}", elem_classes="quiz-topic-btn")
                            topic_btns.append(btn)
                    quiz_state = gr.State()
                    quiz_question_1 = gr.Markdown()
                    quiz_radio_1 = gr.Radio(choices=[], label="")
                    quiz_question_2 = gr.Markdown()
                    quiz_radio_2 = gr.Radio(choices=[], label="")
                    quiz_btn = gr.Button("Submit Quiz")
                    quiz_out = gr.Markdown()

                    def on_topic_select(topic):
                        mcqs = generate_mcqs(topic, n=2)
                        return (f"**Q1:** {mcqs[0]['question']}", mcqs[0]['options'],
                                f"**Q2:** {mcqs[1]['question']}", mcqs[1]['options'], mcqs)
                    for i, btn in enumerate(topic_btns):
                        btn.click(
                            lambda _, t=QUIZ_TOPICS[i]['name']: on_topic_select(t),
                            inputs=btn,
                            outputs=[quiz_question_1, quiz_radio_1, quiz_question_2, quiz_radio_2, quiz_state]
                        )

                    def evaluate_quiz(ans1, ans2, mcqs):
                        correct = 0
                        if ans1 == mcqs[0]['options'][mcqs[0]['answer_idx']]:
                            correct += 1
                        if ans2 == mcqs[1]['options'][mcqs[1]['answer_idx']]:
                            correct += 1
                        user_state.points += correct * 3
                        return f"✅ {correct}/2 Correct. +{correct*3} pts", update_level()
                    quiz_btn.click(evaluate_quiz, [quiz_radio_1, quiz_radio_2, quiz_state], [quiz_out, progress])

                # --- Coding Puzzle Tab ---
                with gr.Tab("💻 Coding Puzzle"):
                    gr.Markdown("<b>Benchmark CPU vs GPU</b><br>Check your cuDF/cuML code accuracy and visualize speedup.")

                    puzzle = {
                        "desc": "Convert Pandas CPU code to cuDF GPU",
                        "cpu": "import pandas as pd\n"
                               "df = pd.DataFrame({'a': [1,2,3]})\n"
                               "print(df.sum())",
                        "gpu": "import cudf\n"
                               "df = cudf.DataFrame({'a': [1,2,3]})\n"
                               "print(df.sum())"
                    }

                    cpu_code = gr.Code(label="CPU Code", value=puzzle['cpu'], interactive=False)
                    user_code = gr.Code(label="Your GPU Code", language="python")
                    check_btn = gr.Button("Check Solution 🚦")
                    bench_btn = gr.Button("Run Benchmark 🚀")
                    feedback = gr.Markdown()
                    bench_out = gr.Markdown()

                    def check_gpu_code(code):
                        import difflib
                        match = difflib.SequenceMatcher(None, code.strip(), puzzle['gpu'].strip()).ratio()
                        if match > 0.7:
                            user_state.points += 5
                            return "🎉 Looks good! +5 points", update_level()
                        else:
                            return "❌ Not quite. Try matching the structure of cuDF.", update_level()
                    def run_benchmark(code):
                        match = difflib.SequenceMatcher(None, code.strip(), puzzle['gpu'].strip()).ratio()
                        if match > 0.7:
                            cpu_time = 1.5
                            gpu_time = 0.2
                            speedup = round(cpu_time / gpu_time, 1)
                            return f"⏱️ CPU time: {cpu_time}s<br>⚡ GPU time: {gpu_time}s<br>🚀 Speedup: {speedup}x"
                        else:
                            return "❌ Code not GPU-convertible. Fix and try again."
                    check_btn.click(check_gpu_code, [user_code], [feedback, progress])
                    bench_btn.click(run_benchmark, [user_code], [bench_out])

    # Optional: CSS for colorful topic buttons
    app.css = """
    .quiz-topic-btn {
        border-radius: 16px !important;
        margin: 8px !important;
        font-size: 1.1em !important;
        padding: 1.2em 2em !important;
        box-shadow: 0 2px 7px #eee;
    }
    """

app.launch(share=True)